## 1. Importação e Configuração Base

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from typing import Tuple, List
import random

# Configuração para reprodutibilidade
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Configuração do ambiente
GRID_SIZE = 10
NUM_STATES = GRID_SIZE * GRID_SIZE
ESTADO_INICIAL = 1
ESTADO_OBJETIVO = 100

PAREDES = {
    23, 24, 25, 26,
    33, 34, 35, 36,
    54, 64, 74, 84,
    55, 65, 75, 85,
}

ACOES = ['UP', 'DOWN', 'LEFT', 'RIGHT']
MAX_PASSOS_POR_EPISODIO = 1000

print("Configuração base importada com sucesso!")

## 2. Funções do Ambiente

In [ ]:
def transicao_estado(estado: int, acao: str) -> int:
    """Calcula o próximo estado após aplicar uma ação."""
    linha = (estado - 1) // GRID_SIZE
    coluna = (estado - 1) % GRID_SIZE
    
    nova_linha, nova_coluna = linha, coluna
    
    if acao == 'UP':
        nova_linha = linha - 1
    elif acao == 'DOWN':
        nova_linha = linha + 1
    elif acao == 'LEFT':
        nova_coluna = coluna - 1
    elif acao == 'RIGHT':
        nova_coluna = coluna + 1
    
    if nova_linha < 0 or nova_linha >= GRID_SIZE or nova_coluna < 0 or nova_coluna >= GRID_SIZE:
        return estado
    
    novo_estado = nova_linha * GRID_SIZE + nova_coluna + 1
    if novo_estado in PAREDES:
        return estado
    
    return novo_estado


def recompensa(estado: int) -> float:
    """Retorna a recompensa de um estado."""
    return 100.0 if estado == ESTADO_OBJETIVO else 0.0


def acao_aleatoria() -> str:
    """Escolhe uma ação aleatória."""
    return np.random.choice(ACOES)


print("Funções do ambiente definidas!")

## 3. Configuração do Q-Learning

In [ ]:
# Parâmetros do Q-Learning
ALPHA = 0.7  # Taxa de aprendizagem
GAMMA = 0.99  # Fator de desconto

# Parâmetros de treino
NUM_PASSOS_TREINO = 20000
NUM_PASSOS_TESTE = 1000
NUM_EXPERIENCIAS = 30

# Pontos de teste durante o treino
PONTOS_TESTE = [100, 200, 500, 600, 700, 800, 900, 1000, 
                2500, 5000, 7500, 10000, 12500, 15000, 17500, 20000]

# Valores de greed a testar
VALORES_GREED = [0.2, 0.5, 0.9]

# Mapeamento de ações para índices
ACAO_PARA_INDICE = {acao: i for i, acao in enumerate(ACOES)}
INDICE_PARA_ACAO = {i: acao for i, acao in enumerate(ACOES)}

print("Configuração do Q-Learning:")
print(f"  α (alpha) = {ALPHA}")
print(f"  γ (gamma) = {GAMMA}")
print(f"  Número de passos de treino: {NUM_PASSOS_TREINO}")
print(f"  Número de experiências: {NUM_EXPERIENCIAS}")
print(f"  Valores de greed a testar: {VALORES_GREED}")

## 4. Funções Auxiliares de Q-Learning

In [ ]:
def inicializar_Q():
    """Inicializa a tabela Q com zeros."""
    return np.zeros((NUM_STATES + 1, len(ACOES)))


def atualizar_Q(Q, estado, acao, proximo_estado, recompensa_recebida):
    """Atualiza a tabela Q usando a equação de Q-Learning."""
    indice_acao = ACAO_PARA_INDICE[acao]
    q_atual = Q[estado, indice_acao]
    max_q_proximo = np.max(Q[proximo_estado, :])
    novo_q = (1 - ALPHA) * q_atual + ALPHA * (recompensa_recebida + GAMMA * max_q_proximo)
    Q[estado, indice_acao] = novo_q


def escolher_melhor_acao(Q, estado):
    """Escolhe a melhor ação com desempate aleatório."""
    valores_q = Q[estado, :]
    max_q = np.max(valores_q)
    acoes_maximas = [i for i, q in enumerate(valores_q) if q == max_q]
    indice_escolhido = np.random.choice(acoes_maximas)
    return INDICE_PARA_ACAO[indice_escolhido]


def escolher_acao_epsilon_greedy(Q, estado, greed):
    """Escolhe ação usando estratégia ε-greedy.
    
    Args:
        Q: Tabela Q
        estado: Estado atual
        greed: Probabilidade de escolher a melhor ação (0 a 1)
    
    Returns:
        Ação escolhida
    """
    if np.random.random() < greed:
        # Exploração: escolhe a melhor ação
        return escolher_melhor_acao(Q, estado)
    else:
        # Exploração: escolhe ação aleatória
        return acao_aleatoria()


def testar_politica_Q(Q, num_passos=NUM_PASSOS_TESTE):
    """Testa a política aprendida sem alterar Q."""
    estado_atual = ESTADO_INICIAL
    recompensa_total = 0.0
    passos_executados = 0
    
    for _ in range(num_passos):
        acao = escolher_melhor_acao(Q, estado_atual)
        proximo_estado = transicao_estado(estado_atual, acao)
        r = recompensa(proximo_estado)
        recompensa_total += r
        passos_executados += 1
        
        if proximo_estado == ESTADO_OBJETIVO:
            proximo_estado = ESTADO_INICIAL
        
        estado_atual = proximo_estado
    
    return recompensa_total / passos_executados if passos_executados > 0 else 0.0


print("Funções de Q-Learning implementadas!")

## 5. Função de Treino ε-greedy com Greed Fixo

In [ ]:
def treino_epsilon_greedy_fixo(greed, seed=None):
    """Executa treino com ε-greedy com valor fixo de greed.
    
    Args:
        greed: Probabilidade fixa de escolher a melhor ação
        seed: Semente para reprodutibilidade
    
    Returns:
        Q: Tabela Q final
        recompensas_teste: Lista de recompensas nos pontos de teste
        tempo_total: Tempo de execução
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    inicio = time.time()
    Q = inicializar_Q()
    estado_atual = ESTADO_INICIAL
    recompensas_teste = []
    
    for passo in range(1, NUM_PASSOS_TREINO + 1):
        # Escolhe ação usando ε-greedy
        acao = escolher_acao_epsilon_greedy(Q, estado_atual, greed)
        
        # Executa ação
        proximo_estado = transicao_estado(estado_atual, acao)
        r = recompensa(proximo_estado)
        
        # Atualiza Q
        atualizar_Q(Q, estado_atual, acao, proximo_estado, r)
        
        # Reset se atingiu objetivo
        if proximo_estado == ESTADO_OBJETIVO:
            proximo_estado = ESTADO_INICIAL
        
        estado_atual = proximo_estado
        
        # Testa política nos pontos definidos
        if passo in PONTOS_TESTE:
            recomp_teste = testar_politica_Q(Q)
            recompensas_teste.append(recomp_teste)
    
    tempo_total = time.time() - inicio
    return Q, recompensas_teste, tempo_total


print("Função de treino ε-greedy (fixo) definida!")

## 6. Função de Treino ε-greedy com Greed Crescente

In [ ]:
def calcular_greed_crescente(passo_atual, total_passos, greed_inicial=0.3, greed_final=1.0):
    """Calcula o valor de greed crescente.
    
    Começa em greed_inicial (30%) e cresce linearmente até greed_final (100%).
    Os primeiros 30% dos passos mantêm greed_inicial.
    
    Args:
        passo_atual: Passo atual do treino
        total_passos: Total de passos de treino
        greed_inicial: Valor inicial de greed (default 0.3)
        greed_final: Valor final de greed (default 1.0)
    
    Returns:
        Valor de greed para o passo atual
    """
    # Primeiros 30% dos passos mantêm greed inicial
    passo_inicio_crescimento = int(0.3 * total_passos)
    
    if passo_atual <= passo_inicio_crescimento:
        return greed_inicial
    
    # Crescimento linear de greed_inicial até greed_final
    progresso = (passo_atual - passo_inicio_crescimento) / (total_passos - passo_inicio_crescimento)
    greed = greed_inicial + progresso * (greed_final - greed_inicial)
    
    return min(greed, greed_final)  # Garante não ultrapassar greed_final


def treino_epsilon_greedy_crescente(seed=None):
    """Executa treino com ε-greedy com greed crescente.
    
    Greed começa em 0.3 (30%) e cresce até 1.0 (100%).
    
    Args:
        seed: Semente para reprodutibilidade
    
    Returns:
        Q: Tabela Q final
        recompensas_teste: Lista de recompensas nos pontos de teste
        tempo_total: Tempo de execução
        valores_greed: Lista de valores de greed ao longo do treino
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    
    inicio = time.time()
    Q = inicializar_Q()
    estado_atual = ESTADO_INICIAL
    recompensas_teste = []
    valores_greed = []
    
    for passo in range(1, NUM_PASSOS_TREINO + 1):
        # Calcula greed crescente
        greed_atual = calcular_greed_crescente(passo, NUM_PASSOS_TREINO)
        
        # Escolhe ação usando ε-greedy
        acao = escolher_acao_epsilon_greedy(Q, estado_atual, greed_atual)
        
        # Executa ação
        proximo_estado = transicao_estado(estado_atual, acao)
        r = recompensa(proximo_estado)
        
        # Atualiza Q
        atualizar_Q(Q, estado_atual, acao, proximo_estado, r)
        
        # Reset se atingiu objetivo
        if proximo_estado == ESTADO_OBJETIVO:
            proximo_estado = ESTADO_INICIAL
        
        estado_atual = proximo_estado
        
        # Testa política nos pontos definidos
        if passo in PONTOS_TESTE:
            recomp_teste = testar_politica_Q(Q)
            recompensas_teste.append(recomp_teste)
            valores_greed.append(greed_atual)
    
    tempo_total = time.time() - inicio
    return Q, recompensas_teste, tempo_total, valores_greed


print("Função de treino ε-greedy (crescente) definida!")

## 7. Experimentos com Greed Fixo (0.2, 0.5, 0.9)

In [ ]:
print("="*80)
print("EXPERIMENTOS COM VALORES FIXOS DE GREED")
print("="*80)

# Dicionário para armazenar resultados
resultados_fixos = {}

for greed in VALORES_GREED:
    print(f"\n{'='*80}")
    print(f"TREINO COM GREED = {greed} ({int(greed*100)}% greedy, {int((1-greed)*100)}% random)")
    print(f"{'='*80}\n")
    
    todas_recompensas = []
    todos_tempos = []
    todas_Q = []
    
    print(f"Executando {NUM_EXPERIENCIAS} experiências...\n")
    
    for exp in range(1, NUM_EXPERIENCIAS + 1):
        Q, recompensas, tempo = treino_epsilon_greedy_fixo(greed, seed=42 + exp)
        todas_recompensas.append(recompensas)
        todos_tempos.append(tempo)
        todas_Q.append(Q.copy())
        
        if exp % 5 == 0:
            print(f"Experiência {exp:2d}: tempo = {tempo:.2f}s, recompensa final = {recompensas[-1]:.6f}")
    
    todas_recompensas = np.array(todas_recompensas)
    todos_tempos = np.array(todos_tempos)
    
    # Armazena resultados
    resultados_fixos[greed] = {
        'recompensas': todas_recompensas,
        'tempos': todos_tempos,
        'Q_final': todas_Q[-1]
    }
    
    print(f"\n{'='*80}")
    print(f"RESULTADOS - GREED = {greed}")
    print(f"{'='*80}")
    print(f"Tempo médio: {np.mean(todos_tempos):.2f}s ± {np.std(todos_tempos):.2f}s")
    print(f"Recompensa final média: {np.mean(todas_recompensas[:, -1]):.6f} ± {np.std(todas_recompensas[:, -1]):.6f}")
    print(f"{'='*80}")

print(f"\n\n{'='*80}")
print("EXPERIMENTOS COM GREED FIXO CONCLUÍDOS!")
print(f"{'='*80}")

## 8. Experimento com Greed Crescente (0.3 → 1.0)

In [ ]:
print("="*80)
print("EXPERIMENTO COM GREED CRESCENTE (0.3 → 1.0)")
print("="*80)
print("\nOs primeiros 30% dos passos mantêm greed = 0.3")
print("Depois, greed cresce linearmente até 1.0 no passo final\n")

todas_recompensas_crescente = []
todos_tempos_crescente = []
todas_Q_crescente = []
todos_valores_greed = []

print(f"Executando {NUM_EXPERIENCIAS} experiências...\n")

for exp in range(1, NUM_EXPERIENCIAS + 1):
    Q, recompensas, tempo, valores_greed = treino_epsilon_greedy_crescente(seed=42 + exp)
    todas_recompensas_crescente.append(recompensas)
    todos_tempos_crescente.append(tempo)
    todas_Q_crescente.append(Q.copy())
    todos_valores_greed.append(valores_greed)
    
    if exp % 5 == 0:
        print(f"Experiência {exp:2d}: tempo = {tempo:.2f}s, recompensa final = {recompensas[-1]:.6f}")

todas_recompensas_crescente = np.array(todas_recompensas_crescente)
todos_tempos_crescente = np.array(todos_tempos_crescente)
valores_greed_medios = np.mean(todos_valores_greed, axis=0)

print(f"\n{'='*80}")
print("RESULTADOS - GREED CRESCENTE")
print(f"{'='*80}")
print(f"Tempo médio: {np.mean(todos_tempos_crescente):.2f}s ± {np.std(todos_tempos_crescente):.2f}s")
print(f"Recompensa final média: {np.mean(todas_recompensas_crescente[:, -1]):.6f} ± {np.std(todas_recompensas_crescente[:, -1]):.6f}")
print(f"Greed inicial: {valores_greed_medios[0]:.3f}")
print(f"Greed final: {valores_greed_medios[-1]:.3f}")
print(f"{'='*80}")

## 9. Visualização: Comparação de Todos os Métodos

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

# Cores para cada método
cores = {
    0.2: 'red',
    0.5: 'orange',
    0.9: 'blue',
    'crescente': 'green'
}

# Plot greed fixo
for greed in VALORES_GREED:
    recompensas = resultados_fixos[greed]['recompensas']
    medias = np.mean(recompensas, axis=0)
    desvios = np.std(recompensas, axis=0)
    
    label = f'Greed = {greed} ({int(greed*100)}% greedy)'
    ax.plot(PONTOS_TESTE, medias, color=cores[greed], linewidth=2.5, 
            marker='o', markersize=7, label=label, zorder=3)
    ax.fill_between(PONTOS_TESTE, medias - desvios, medias + desvios, 
                    alpha=0.15, color=cores[greed])

# Plot greed crescente
medias_crescente = np.mean(todas_recompensas_crescente, axis=0)
desvios_crescente = np.std(todas_recompensas_crescente, axis=0)

ax.plot(PONTOS_TESTE, medias_crescente, color=cores['crescente'], linewidth=3, 
        marker='D', markersize=8, label='Greed Crescente (0.3→1.0)', zorder=4)
ax.fill_between(PONTOS_TESTE, medias_crescente - desvios_crescente, 
                medias_crescente + desvios_crescente, 
                alpha=0.15, color=cores['crescente'])

ax.set_xlabel('Número de Passos de Treino', fontsize=14, fontweight='bold')
ax.set_ylabel('Recompensa Média por Passo (Teste)', fontsize=14, fontweight='bold')
ax.set_title('Exercício 3: Comparação de Estratégias ε-greedy\n'
             f'(α={ALPHA}, γ={GAMMA}, {NUM_EXPERIENCIAS} experiências)', 
             fontsize=15, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(fontsize=11, loc='lower right', framealpha=0.95)

plt.tight_layout()
plt.savefig('exercicio3_comparacao_epsilon_greedy.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico salvo como 'exercicio3_comparacao_epsilon_greedy.png'")

## 10. Visualização: Evolução do Greed Crescente

In [ ]:
# Criar array de todos os passos para visualização
todos_passos = np.arange(1, NUM_PASSOS_TREINO + 1)
todos_greeds = [calcular_greed_crescente(p, NUM_PASSOS_TREINO) for p in todos_passos]

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Subplot 1: Evolução do greed
ax1.plot(todos_passos, todos_greeds, color='purple', linewidth=2.5)
ax1.axhline(y=0.3, color='red', linestyle='--', linewidth=1.5, 
            label='Greed inicial (0.3)', alpha=0.7)
ax1.axhline(y=1.0, color='green', linestyle='--', linewidth=1.5, 
            label='Greed final (1.0)', alpha=0.7)
ax1.axvline(x=0.3*NUM_PASSOS_TREINO, color='orange', linestyle=':', 
            linewidth=2, label='30% dos passos', alpha=0.7)

ax1.set_xlabel('Passo de Treino', fontsize=13, fontweight='bold')
ax1.set_ylabel('Valor de Greed (ε)', fontsize=13, fontweight='bold')
ax1.set_title('Evolução do Greed ao Longo do Treino', fontsize=14, fontweight='bold', pad=15)
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=11, loc='right')
ax1.set_ylim([0, 1.1])

# Subplot 2: Comparação com greed fixo nos pontos de teste
medias_crescente = np.mean(todas_recompensas_crescente, axis=0)
medias_09 = np.mean(resultados_fixos[0.9]['recompensas'], axis=0)
medias_05 = np.mean(resultados_fixos[0.5]['recompensas'], axis=0)

ax2.plot(PONTOS_TESTE, medias_crescente, 'g-', linewidth=3, 
         marker='D', markersize=8, label='Greed Crescente (0.3→1.0)', zorder=3)
ax2.plot(PONTOS_TESTE, medias_09, 'b--', linewidth=2, 
         marker='o', markersize=6, label='Greed = 0.9 (fixo)', alpha=0.7)
ax2.plot(PONTOS_TESTE, medias_05, 'orange', linewidth=2, linestyle='--',
         marker='s', markersize=6, label='Greed = 0.5 (fixo)', alpha=0.7)

ax2.set_xlabel('Número de Passos de Treino', fontsize=13, fontweight='bold')
ax2.set_ylabel('Recompensa Média por Passo', fontsize=13, fontweight='bold')
ax2.set_title('Desempenho: Greed Crescente vs Greed Fixo', fontsize=14, fontweight='bold', pad=15)
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=11, loc='lower right')

plt.tight_layout()
plt.savefig('exercicio3_greed_crescente_detalhe.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico salvo como 'exercicio3_greed_crescente_detalhe.png'")

## 11. Mapas de Calor das Utilidades Aprendidas

In [ ]:
def criar_mapa_calor_utilidade(Q, titulo="Mapa de Calor da Utilidade", 
                               nome_ficheiro="mapa_calor.png"):
    """Cria mapa de calor mostrando U(s) = max_a Q[s,a]."""
    utilidades = np.zeros((GRID_SIZE, GRID_SIZE))
    
    for estado in range(1, NUM_STATES + 1):
        linha = (estado - 1) // GRID_SIZE
        coluna = (estado - 1) % GRID_SIZE
        utilidades[linha, coluna] = np.max(Q[estado, :])
    
    for estado in PAREDES:
        linha = (estado - 1) // GRID_SIZE
        coluna = (estado - 1) % GRID_SIZE
        utilidades[linha, coluna] = np.nan
    
    fig, ax = plt.subplots(figsize=(12, 11))
    im = ax.imshow(utilidades, cmap='YlOrRd', interpolation='nearest')
    
    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Utilidade U(s) = max Q[s,a]', fontsize=12, fontweight='bold')
    
    for estado in range(1, NUM_STATES + 1):
        linha = (estado - 1) // GRID_SIZE
        coluna = (estado - 1) % GRID_SIZE
        
        if estado in PAREDES:
            ax.text(coluna, linha, 'X', ha='center', va='center',
                   color='black', fontsize=14, fontweight='bold')
        else:
            utilidade = utilidades[linha, coluna]
            cor_texto = 'white' if utilidade > np.nanmax(utilidades) * 0.5 else 'black'
            ax.text(coluna, linha, f'{estado}\n{utilidade:.1f}', 
                   ha='center', va='center', color=cor_texto, 
                   fontsize=8, fontweight='bold')
    
    ax.set_xticks(range(GRID_SIZE))
    ax.set_yticks(range(GRID_SIZE))
    ax.set_xticklabels(range(1, GRID_SIZE + 1))
    ax.set_yticklabels(range(1, GRID_SIZE + 1))
    ax.set_xlabel('Coluna', fontsize=12, fontweight='bold')
    ax.set_ylabel('Linha', fontsize=12, fontweight='bold')
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    
    ax.set_xticks(np.arange(-0.5, GRID_SIZE, 1), minor=True)
    ax.set_yticks(np.arange(-0.5, GRID_SIZE, 1), minor=True)
    ax.grid(which='minor', color='gray', linestyle='-', linewidth=1)
    ax.tick_params(which='minor', size=0)
    
    plt.tight_layout()
    plt.savefig(nome_ficheiro, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Mapa de calor salvo como '{nome_ficheiro}'")


print("Criando mapas de calor...\n")

# Mapas para valores fixos de greed
for greed in VALORES_GREED:
    criar_mapa_calor_utilidade(
        resultados_fixos[greed]['Q_final'],
        titulo=f"Mapa de Calor - ε-greedy com greed={greed}",
        nome_ficheiro=f"exercicio3_mapa_calor_greed_{int(greed*10)}.png"
    )

# Mapa para greed crescente
criar_mapa_calor_utilidade(
    todas_Q_crescente[-1],
    titulo="Mapa de Calor - ε-greedy com greed crescente (0.3→1.0)",
    nome_ficheiro="exercicio3_mapa_calor_greed_crescente.png"
)

## 12. Análise Estatística Comparativa

In [ ]:
print("="*90)
print("ANÁLISE ESTATÍSTICA COMPARATIVA - EXERCÍCIO 3")
print("="*90)

print("\n" + "="*90)
print("1. TEMPO DE EXECUÇÃO")
print("="*90)

for greed in VALORES_GREED:
    tempos = resultados_fixos[greed]['tempos']
    print(f"Greed = {greed:3.1f}: {np.mean(tempos):6.2f}s ± {np.std(tempos):5.2f}s")

print(f"Crescente:   {np.mean(todos_tempos_crescente):6.2f}s ± {np.std(todos_tempos_crescente):5.2f}s")

print("\n" + "="*90)
print("2. RECOMPENSA FINAL (após 20000 passos)")
print("="*90)

resultados_finais = {}

for greed in VALORES_GREED:
    recomp_final = resultados_fixos[greed]['recompensas'][:, -1]
    media = np.mean(recomp_final)
    desvio = np.std(recomp_final)
    resultados_finais[greed] = media
    print(f"Greed = {greed:3.1f}: {media:.6f} ± {desvio:.6f}")

recomp_final_crescente = todas_recompensas_crescente[:, -1]
media_crescente = np.mean(recomp_final_crescente)
desvio_crescente = np.std(recomp_final_crescente)
resultados_finais['crescente'] = media_crescente
print(f"Crescente:   {media_crescente:.6f} ± {desvio_crescente:.6f}")

print("\n" + "="*90)
print("3. COMPARAÇÃO DE DESEMPENHO")
print("="*90)

melhor_metodo = max(resultados_finais, key=resultados_finais.get)
melhor_valor = resultados_finais[melhor_metodo]

print(f"\nMelhor método: {melhor_metodo if isinstance(melhor_metodo, str) else f'Greed = {melhor_metodo}'}")
print(f"Recompensa: {melhor_valor:.6f}\n")

print("Melhoria relativa em relação aos outros métodos:")
for metodo, valor in sorted(resultados_finais.items(), key=lambda x: x[1], reverse=True):
    if metodo != melhor_metodo:
        melhoria = ((melhor_valor - valor) / valor * 100)
        nome_metodo = metodo if isinstance(metodo, str) else f'Greed = {metodo}'
        print(f"  vs {nome_metodo:15s}: {melhoria:+6.2f}%")

print("\n" + "="*90)
print("4. TAXA DE EXPLORAÇÃO vs EXPLORAÇÃO")
print("="*90)

for greed in VALORES_GREED:
    exploracao = int((1 - greed) * 100)
    exploracao_word = int(greed * 100)
    print(f"Greed = {greed}: {exploracao_word}% exploração (greedy), {exploracao}% exploração (random)")

print(f"Crescente:   30% exploração inicial → 100% exploração final")

print("\n" + "="*90)

## 13. Boxplot Comparativo de Recompensas Finais

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

# Preparar dados para boxplot
dados_boxplot = []
labels = []

for greed in VALORES_GREED:
    recomp_final = resultados_fixos[greed]['recompensas'][:, -1]
    dados_boxplot.append(recomp_final)
    labels.append(f'Greed={greed}\n({int(greed*100)}% greedy)')

dados_boxplot.append(todas_recompensas_crescente[:, -1])
labels.append('Greed\nCrescente\n(0.3→1.0)')

bp = ax.boxplot(dados_boxplot, labels=labels, patch_artist=True,
                notch=True, showmeans=True,
                meanprops=dict(marker='D', markerfacecolor='red', markersize=8),
                medianprops=dict(color='darkblue', linewidth=2))

# Colorir boxes
cores_boxes = ['#ffcccc', '#ffcc99', '#ccccff', '#ccffcc']
for patch, cor in zip(bp['boxes'], cores_boxes):
    patch.set_facecolor(cor)
    patch.set_alpha(0.7)

ax.set_ylabel('Recompensa Média por Passo (após 20000 passos)', fontsize=13, fontweight='bold')
ax.set_title('Exercício 3: Comparação de Desempenho Final\n'
             f'Estratégias ε-greedy ({NUM_EXPERIENCIAS} experiências)',
             fontsize=14, fontweight='bold', pad=20)
ax.grid(True, axis='y', alpha=0.3, linestyle='--')

# Adicionar legenda
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='white', edgecolor='black', label='Mediana (linha azul)'),
    plt.Line2D([0], [0], marker='D', color='w', markerfacecolor='red', 
               markersize=8, label='Média (diamante vermelho)')
]
ax.legend(handles=legend_elements, fontsize=11, loc='lower right')

plt.tight_layout()
plt.savefig('exercicio3_boxplot_comparacao.png', dpi=300, bbox_inches='tight')
plt.show()

print("Gráfico salvo como 'exercicio3_boxplot_comparacao.png'")

## 14. Conclusões do Exercício 3

### Análise dos Resultados

#### 1. Greed = 0.2 (20% greedy, 80% random)

**Características:**
- Alta exploração (80% ações aleatórias)
- Baixa exploração (20% melhor ação)

**Resultados:**
- Convergência lenta
- Muita exploração do espaço de estados
- Desempenho final limitado pela aleatoriedade excessiva
- Similar ao Random Walk puro

#### 2. Greed = 0.5 (50% greedy, 50% random)

**Características:**
- Equilíbrio entre exploração e exploração
- 50% de cada estratégia

**Resultados:**
- Convergência moderada
- Boa exploração do espaço
- Desempenho intermediário
- Bom compromisso para ambientes desconhecidos

#### 3. Greed = 0.9 (90% greedy, 10% random)

**Características:**
- Alta exploração (90% melhor ação)
- Baixa exploração (10% ações aleatórias)

**Resultados:**
- Convergência rápida
- Exploração focada
- Melhor desempenho entre os valores fixos
- Mínima aleatoriedade mantém descoberta de alternativas

#### 4. Greed Crescente (0.3 → 1.0)

**Características:**
- **Fase inicial (primeiros 30%)**: greed = 0.3 (exploração)
- **Fase crescente (70% restantes)**: greed aumenta linearmente até 1.0 (exploração)

**Estratégia:**
```
Passos 1-6000:      greed = 0.3  (30% greedy, 70% random)
Passos 6001-20000:  greed: 0.3 → 1.0 (crescimento linear)
Passo 20000:        greed = 1.0  (100% greedy, 0% random)
```

**Resultados:**
- **Melhor desempenho global**
- Combina vantagens de ambas as fases:
  - Exploração inicial ampla
  - Exploração progressiva à medida que aprende
- Convergência final muito forte
- Tabela Q mais robusta

### Comparação Final

**Ranking de Desempenho (do melhor ao pior):**

1. **Greed Crescente** (0.3 → 1.0)
2. **Greed = 0.9** (fixo)
3. **Greed = 0.5** (fixo)
4. **Greed = 0.2** (fixo)

### Lições Aprendidas

1. **Dilema Exploração-Exploração:**
   - Muita exploração → convergência lenta
   - Muita exploração → pode ficar presa em mínimos locais
   - **Solução:** estratégia adaptativa

2. **Estratégia Crescente é Superior:**
   - Explora amplamente no início
   - Refina progressivamente
   - Converge para política ótima

3. **Importância da Exploração Inicial:**
   - Primeiros 30% com exploração alta garante descoberta
   - Evita convergência prematura

4. **Transição Suave:**
   - Crescimento linear permite adaptação gradual
   - Não há mudanças bruscas de comportamento

### Aplicações Práticas

A estratégia **ε-greedy com greed crescente** é amplamente usada em:
- Robótica (aprendizagem de controlo)
- Jogos (IA adaptativa)
- Sistemas de recomendação
- Otimização de recursos

### Próximos Passos

Melhorias possíveis:
- Testar diferentes curvas de crescimento (exponencial, logarítmica)
- Variar o ponto de início do crescimento (testar 20%, 40%, etc.)
- Implementar decay exponencial: $\epsilon_t = \epsilon_0 \cdot e^{-\lambda t}$
- Explorar estratégias adaptativas baseadas no desempenho